In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(33, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    author_id = row.author_id
    
    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)
    
    test = "batch10" # change before run
    
    level = "word"
    
    iterations = 30
    
    dropout = 0.4
    
    samples = 7000
    
    dimensions = 200
    
    loc = authorList.index(author_id)
    
    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))
    
    print("Current test: %s" % (str(printstate)))
    
    import UpdateDB as db
    
    case = db.checkOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions, samples = samples, 
                              iterations = iterations, dropout = dropout, test = test)
    
    if case == False:
        
        print("Running: %12s" % (str(printstate)))
        
        import StyloNeural as Stylo
        (labels_index, predYList, predY, history, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], 
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
        
        for key, auth in labels_index.iteritems():
            if auth == author_id:
                loc = key
        
        ans = predY[loc]
        
        import UpdateDB as db
        case = db.updateresultOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions, 
                                         samples = samples, iterations = iterations, dropout = dropout, 
                                         accuracy = ans, test = test)
        
        if case:
            ans = 0.0

            if (predY.tolist().index(max(predY)) == loc):
                ans = 1.0

            import UpdateDB as db
            case = db.updateresultOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions, 
                                             samples = samples, iterations = iterations, dropout = dropout, 
                                             accuracy = ans, test = str(test + 'binary'))
            
        else:
            print("Skipped: %12s" % (str(printstate)))
        
        del Stylo
        
        from keras import backend as K
        K.clear_session()
        
        import time
        time.sleep(10)
        
        from IPython.display import clear_output
        clear_output()
    
    else:
        print("Skipped: %12s" % (str(printstate)))
                

Current test: doc_id = 227, candidate = 3, dimensions = 200, samples = 7000, 
iterations = 30, dropout = 0.4, test = batch10
Execution completed
Running: doc_id = 227, candidate = 3, dimensions = 200, samples = 7000, 
iterations = 30, dropout = 0.4, test = batch10
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 156
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (16109, 2)
Author:    23  Size:  6082
Author:    59  Size:  4234
Author:    88  Size:  5793
Min: 4234
Max: 6082
Authors [23, 59, 88].
Found 12702 texts.
Found 12702 labels.
Found 95153 unique tokens.
Shape of data tensor: (12702, 1000)
Shape of label tensor: (12702, 3)
Done compiling.
Train on 9144 samples, validate on 2287 samples
Epoch 1/30
9144/9144 [==============================] - 95s - loss: 0.8292 - acc: 0.6087 - val_loss: 0.5569 - val_acc: 0.7949

In [ ]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
%tb